In [1]:
from mpi4py import MPI
import wildmeshing
import dolfinx
from pathlib import Path
import json

In [2]:
folder = Path("../../stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/")
assert folder.exists(), "Could not find stl files"
tree = {"operation":"union",
        "left": str((folder/"ventricles.final.stl").absolute().as_posix()),
        "right": str((folder/"rhchoroid.plexus.final.stl").absolute().as_posix())
       }
tetra = wildmeshing.Tetrahedralizer(stop_quality=100, max_its=30, edge_length_r=0.1)
tetra.load_csg_tree(json.dumps(tree))

TBB threads 16
o-[I/O         ] Loading file /root/shared/src/ucs/../../stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/ventricles.final.stl...
                 (FP64) nb_v:134064 nb_e:0 nb_f:44688 nb_b:134064 tri:1 dim:3
                 Attributes on vertices: point[3]
                 Loading file /root/shared/src/ucs/../../stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/rhchoroid.plexus.final.stl...
                 (FP64) nb_v:7908 nb_e:0 nb_f:2636 nb_b:7908 tri:1 dim:3
                 Attributes on vertices: point[3]
bbox_diag_length = 145.674
ideal_edge_length = 14.5674
stage = 2
eps_input = 0.145674
eps = 0.0806437
eps_simplification = 0.064515
eps_coplanar = 0.000145674
dd = 0.0971161
dd_simplification = 0.0776929


In [3]:
# Create mesh
success = tetra.tetrahedralize()
print(success)

None
[2025-02-17 08:38:50.200] [float-tetwild] [info] remove duplicates: 
[2025-02-17 08:38:50.200] [float-tetwild] [info] #v: 141972 -> 23655
[2025-02-17 08:38:50.200] [float-tetwild] [info] #f: 47324 -> 47324
collapsing 1.01889
swapping 0.014188
[2025-02-17 08:38:51.239] [float-tetwild] [info] remove duplicates: 
[2025-02-17 08:38:51.239] [float-tetwild] [info] #v: 6583 -> 6583
[2025-02-17 08:38:51.239] [float-tetwild] [info] #f: 13180 -> 13180
[2025-02-17 08:38:51.239] [float-tetwild] [info] #v = 6583
[2025-02-17 08:38:51.239] [float-tetwild] [info] #f = 13179
#boundary_e1 = 1
#boundary_e2 = 2
[2025-02-17 08:38:51.250] [float-tetwild] [info] preprocessing 1.07112s
[2025-02-17 08:38:51.250] [float-tetwild] [info] 
[2025-02-17 08:38:51.306] [float-tetwild] [info] #v = 8433
[2025-02-17 08:38:51.306] [float-tetwild] [info] #t = 50157
[2025-02-17 08:38:51.306] [float-tetwild] [info] tetrahedralizing 0.056272s
[2025-02-17 08:38:51.306] [float-tetwild] [info] 
[2025-02-17 08:38:51.306] [fl

In [4]:
point_array, cell_array, marker = tetra.get_tet_mesh()

[2025-02-17 08:38:57.342] [float-tetwild] [info] after winding number
[2025-02-17 08:38:57.342] [float-tetwild] [info] #v = 5371
[2025-02-17 08:38:57.342] [float-tetwild] [info] #t = 15099
[2025-02-17 08:38:57.342] [float-tetwild] [info] winding number 1.73978e+09s
[2025-02-17 08:38:57.342] [float-tetwild] [info] 


In [5]:
point_array

array([[-44.29054371, -24.08801509,  -4.01255037],
       [-44.25918768, -25.76470236,  -4.15634641],
       [-44.26692117, -25.26247254,  -2.94195196],
       ...,
       [ 26.44556283, -56.011846  ,  10.69396741],
       [-21.49479902, -29.96351311,  13.81143336],
       [ -7.7784267 , -18.77944804,  14.97208435]])

In [6]:
cell_array

array([[3077, 3049, 3115, 3119],
       [2961, 2984, 3007, 3064],
       [4204, 4248, 4222, 4298],
       ...,
       [2156, 1908, 2055, 2118],
       [ 675,  250,  651,  589],
       [4834,  589,  250,  675]], dtype=int32)

In [7]:
marker

array([[1.],
       [2.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [8]:
import numpy as np
np.unique(marker)

array([1., 2.])

In [9]:
import pyvista
import ufl
import basix.ufl

In [10]:
mesh = dolfinx.mesh.create_mesh(MPI.COMM_WORLD, cell_array, point_array, ufl.Mesh(basix.ufl.element("Lagrange", "tetrahedron", 1, shape=(3,))))

In [11]:
import pyvista
pv_grid = pyvista.UnstructuredGrid(*dolfinx.plot.vtk_mesh(mesh))

In [12]:
pyvista.start_xvfb(1.0)
pyvista.set_jupyter_backend("trame")
pyvista.off_screen=False
plotter = pyvista.Plotter()
plotter.add_mesh(pv_grid)
plotter.show()

error: XDG_RUNTIME_DIR is invalid or not set in the environment.
MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


Widget(value='<iframe src="http://localhost:35983/index.html?ui=P_0x79777d7df560_0&reconnect=auto" class="pyvi…